## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
#from google.colab import drive  
#drive.mount('/content/drive/')

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/gdrive/My Drive/Colab Notebooks/CNN/DogBreed_Classification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

In [0]:
import pandas as pd

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
doglabels = pd.read_csv('labels.csv')

In [0]:
doglabels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [0]:
doglabels['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
brabancon_griffon        67
komondor                 67
golden_retriever         67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
import tensorflow.keras

In [0]:
dfDummies = pd.get_dummies(doglabels['breed'])

In [0]:
dfDummies.shape

(10222, 120)

In [0]:
dfDummies.head()

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
img_rows = 128
img_cols = 128

In [0]:
x_train = []
y_train = []
loc = []

from tqdm import tqdm
import cv2
for i, img in tqdm(doglabels.values):
  train_image = cv2.imread('./train/{}.jpg'.format(i),3)
  loc.append('./train/'+ i + '.jpg')
  tr_img_resize = cv2.resize(train_image,(img_rows,img_cols))
  x_train.append(tr_img_resize)
  y_train.append(img)

100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [00:23<00:00, 430.46it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
import numpy as np
x_train_data = np.array(x_train)
y_train_data = np.array(y_train)


In [0]:
x_train_data.shape,y_train_data.shape

((10222, 128, 128, 3), (10222,))

In [0]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [0]:
y_train_data = pd.get_dummies(y_train_data)

In [0]:
y_train_data.shape

(10222, 120)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split
Xtrain,Xval,ytrain,yval = train_test_split(x_train_data,y_train_data, test_size = 0.20)
Xtrain.shape,Xval.shape,ytrain.shape,yval.shape

((8177, 128, 128, 3), (2045, 128, 128, 3), (8177, 120), (2045, 120))

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img = pd.read_csv("sample_submission.csv",usecols = [0])
test_img.head(2)

,id
0,000621fb3cbb32d8935728e48679680e
1,00102ee9d8eb90812350685311fe5890


In [0]:
test_img.shape

(10357, 1)

Run the below code to load the test image files in x_test_feature

In [0]:
x_test_feature = []
#i = 0 # initialisation
for i in tqdm(test_img['id']): # f for format ,jpg    
    img = cv2.imread('./test/{}.jpg'.format(i), 1)  
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|███████████████████████████████████████████████████████████████████████████| 10357/10357 [00:26<00:00, 388.96it/s]


Normalize the test data and convert it into 4 dimensions

In [0]:
x_test_data = np.array(x_test_feature).astype("float")

In [0]:
x_test_data /= 255

In [0]:
x_test_data.shape

(10357, 128, 128, 3)

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Activation,Flatten
from tensorflow.keras.layers import Convolution2D
 

In [0]:
Model1 = Sequential()
# 1st Conv Layer
Model1.add(Convolution2D(136, 5, 5, input_shape=(128, 128, 3)))
Model1.add(Activation('relu'))
# 2nd Conv Layer
Model1.add(Convolution2D(120, 3, 3))
Model1.add(Activation('relu'))
#Dense layer
Model1.add(Flatten())
Model1.add(Dense(256))
Model1.add(Activation('relu'))
#final layer
Model1.add(Dense(120))
Model1.add(Activation('softmax'))
Model1.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 136)       10336     
_________________________________________________________________
activation (Activation)      (None, 25, 25, 136)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 120)         147000    
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 120)         0         
_________________________________________________________________
flatten (Flatten)            (None, 7680)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1966336   
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0

### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:
# Loss and Optimizer
Model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Store Training Results
early_stopping = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
callback_list = [early_stopping]
   

In [0]:
batch_size = 128
epochs = 10
# Train the model
Model1.fit(Xtrain, ytrain, epochs=epochs, batch_size=batch_size,
              validation_data=(Xval, yval), callbacks=callback_list, verbose=True)
loss_and_metrics = Model1.evaluate(Xval, yval)
print(loss_and_metrics)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 4s 438us/sample - loss: 8.1637 - accuracy: 0.0110 - val_loss: 4.7854 - val_accuracy: 0.0156
Epoch 2/10
8177/8177 [==============================] - 2s 216us/sample - loss: 4.7439 - accuracy: 0.0236 - val_loss: 4.7911 - val_accuracy: 0.0142
Epoch 3/10
8177/8177 [==============================] - 2s 217us/sample - loss: 4.5353 - accuracy: 0.0500 - val_loss: 4.8452 - val_accuracy: 0.0191
Epoch 4/10
8177/8177 [==============================] - 2s 215us/sample - loss: 4.1265 - accuracy: 0.1180 - val_loss: 5.0920 - val_accuracy: 0.0244
Epoch 5/10
8177/8177 [==============================] - 2s 217us/sample - loss: 3.4875 - accuracy: 0.2362 - val_loss: 5.7498 - val_accuracy: 0.0225
Epoch 6/10
8177/8177 [==============================] - 2s 217us/sample - loss: 2.7814 - accuracy: 0.3796 - val_loss: 7.1195 - val_accuracy: 0.0220
Epoch 7/10
8177/8177 [==============================] - 2s 217us

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
# This will do preprocessing and realtime data augmentation:
train_datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [0]:
# This will do preprocessing and realtime data augmentation:
test_datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [0]:
Model1.fit_generator(train_datagen.flow(Xtrain, ytrain,batch_size=32),
                    epochs=10,
                    validation_data=test_datagen.flow(Xval, yval,batch_size=32), callbacks=callback_list)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 256 steps, validate for 64 steps
Epoch 1/10
256/256 [==============================] - 30s 118ms/step - loss: 4.9106 - accuracy: 0.0126 - val_loss: 4.7885 - val_accuracy: 0.0108
Epoch 2/10
256/256 [==============================] - 29s 113ms/step - loss: 4.7807 - accuracy: 0.0127 - val_loss: 4.7864 - val_accuracy: 0.0112
Epoch 3/10
256/256 [==============================] - 30s 116ms/step - loss: 4.7789 - accuracy: 0.0126 - val_loss: 4.7862 - val_accuracy: 0.0108
Epoch 4/10
256/256 [==============================] - 31s 123ms/step - loss: 4.7776 - accuracy: 0.0127 - val_loss: 4.7890 - val_accuracy: 0.0108
Epoch 5/10
256/256 [==============================] - 31s 120ms/step - loss: 4.7780 - accuracy: 0.0136 - val_loss: 4.7912 - val_accuracy: 0.0142
Epoch 6/10
256/256 [==============================] - 30s 118ms/step - loss: 4.7756 - accuracy: 0.0137 - v

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
#base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
base_model= VGG16(weights=('./'+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg',input_shape = (128, 128, 3))

Print the summary of the base_model

In [0]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:

#Adding custom Layers 
Model1 = base_model.output
#Dense layer
Model2 = Flatten()(Model1)
Model2 = Dense(1024,activation = 'relu')(Model2)
Model2 = Dense(256,activation = 'relu')(Model2)
pred = Dense(120,activation = 'softmax')(Model2)


# creating the final model 
Tran_Model1 = Model(base_model.input,pred)

# Loss and Optimizer
Tran_Model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


   

In [0]:
Tran_Model1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
for layer in base_model.layers:
    layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
batch_size = 128
epochs = 10
# Train the model

Tran_Model1.fit(Xtrain, ytrain, epochs=epochs, batch_size=batch_size,
              validation_data=(Xval, yval), callbacks=callback_list, verbose=True)
loss_and_metrics = Tran_Model1.evaluate(Xval, yval)
print(loss_and_metrics)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 108s 13ms/sample - loss: 4.7877 - accuracy: 0.0122 - val_loss: 4.7858 - val_accuracy: 0.0108
Epoch 2/10
8177/8177 [==============================] - 99s 12ms/sample - loss: 4.7842 - accuracy: 0.0126 - val_loss: 4.7849 - val_accuracy: 0.0108
Epoch 3/10
8177/8177 [==============================] - 261s 32ms/sample - loss: 4.7818 - accuracy: 0.0127 - val_loss: 4.7848 - val_accuracy: 0.0108
Epoch 4/10
8177/8177 [==============================] - 296s 36ms/sample - loss: 4.7793 - accuracy: 0.0127 - val_loss: 4.7856 - val_accuracy: 0.0108
Epoch 5/10
8177/8177 [==============================] - 298s 36ms/sample - loss: 4.7777 - accuracy: 0.0127 - val_loss: 4.7869 - val_accuracy: 0.0108
Epoch 6/10
8177/8177 [==============================] - 296s 36ms/sample - loss: 4.7767 - accuracy: 0.0127 - val_loss: 4.7886 - val_accuracy: 0.0108
Epoch 7/10
8177/8177 [==============================] - 299